In [ ]:
import gurobipy as gb
from numpy import linalg as LA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from PositionSelector import *


In [ ]:
"""71个分拣中心的坐标"""
number_sort = 71
sorting_index = 100 * \
    np.random.RandomState(152).random(size=(number_sort, 2))  # 分拣中心的坐标


In [ ]:
"""计算路径并且确认"""
chengben  = list()
for xun in range(20):
    xiaoqu = pd.read_csv('Temp\XiaoQu_Cluster.csv')
    p = xiaoqu['需求'].tolist()
    q = 0.2
    v = 100000 * 0.4
    xiaoqu_index = np.array(xiaoqu[['横坐标', '纵坐标']])
    distance = list()
    for i in range(sorting_index.shape[0]):
        sorting_i = np.array([sorting_index[i]])
        res = LA.norm(np.array([sorting_i[:, 0] - xiaoqu_index[:, 0], sorting_i[:, 1] - xiaoqu_index[:, 1]]), axis=0)
        distance.append(res.tolist())
    distance = np.array(distance)*0.32  # 比例尺问题
    model = gb.Model()
    w = model.addVars(number_sort, 1409, lb=0.0, ub=40000,
                    vtype=gb.GRB.CONTINUOUS, name='w')  # 添加变量
    model.setObjective(gb.quicksum(w[i, j] * distance[i, j] * q for i in range(number_sort)
                                for j in range(1409)), gb.GRB.MINIMIZE)  # 添加目标函数
    model.addConstrs(w.sum('*', j) == p[j] for j in range(1409))
    model.addConstrs(w.sum(i, '*') <= v for i in range(number_sort))
    model.Params.LogToConsole = True  # 显示求解过程
    model.Params.TimeLimit = 100  # 限制求解时间为 100s
    model.optimize()
    cost = model.objVal
    """更新坐标"""
    for i in range(number_sort):
        sorting_index[i, 0] = (sum(w[i, j].X * q * xiaoqu_index[j, 0]/distance[i, j] for j in range(
            1409))+0.0000001) / (sum(w[i, j].X * q / distance[i, j] for j in range(1409))+0.0000001) + 0.1*np.random.RandomState(152).random()
        sorting_index[i, 1] = (sum(w[i, j].X * q * xiaoqu_index[j, 1]/distance[i, j] for j in range(
            1409))+0.0000001) / (sum(w[i, j].X * q / distance[i, j] for j in range(1409))+0.000001) + 0.1*np.random.RandomState(152).random()
    chengben.append(cost)
print("总花费为",cost)


In [ ]:
plt.plot(chengben)

In [ ]:
all_renkou = pd.read_csv('Temp\XiaoQu_Cluster.csv')['小区人口数'].tolist()
all_xuqiu = pd.read_csv('Temp\XiaoQu_Cluster.csv')['需求'].tolist()
all_quyu = pd.read_csv('Temp\XiaoQu_Cluster.csv')['所属区域'].tolist()
give_plan = list()
row_index = list()
column_index = list()
distance_plan = list()
Population = list()
area = list()
for i in range(number_sort):
    give = sum(w[i,j].X for j in range(1409))
    if give!=0:
        row_index.append(sorting_index[i,0])
        column_index.append(sorting_index[i,1])
        give_plan.append([w[i,j].X for j in range(1409)])
        distance_plan.append([distance[i,j] for j in range(1409)])
give_plan = np.array(give_plan)
distance_plan = np.array(distance_plan)
xiaoqugeshu = list()
xuanzhibanjing = list()
for i in range(39):
    count = 0
    banjing = list()
    renkou = list()
    quyu = list()
    for j in range(1409):
        give = give_plan[i, j]
        if give != 0:
            count += 1
            banjing.append(distance_plan[i, j])
            renkou.append(all_renkou[j]*give/all_xuqiu[j])
            quyu.append(all_quyu[j])
    xiaoqugeshu.append(count)
    xuanzhibanjing.append(max(banjing))
    Population.append(int(sum(renkou)))
    area.append(max(set(quyu), key=quyu.count))


In [ ]:
result = pd.DataFrame({
    '横坐标':row_index,
    '纵坐标':column_index,
    '选址半径(km)':xuanzhibanjing,
    '辖区小区数':xiaoqugeshu,
    '辖区内人口数':Population,
    '所在区域':area
})
result.to_csv('Result/物资分发中心坐标.csv',index=False)
result.to_excel('Result/物资分发中心坐标.xlsx', index=False)